### 데이터 균형 맞추기

In [3]:
import pandas as pd
import os

DATA_ROOT = r"D:\학교\3-2\파이썬기반 딥러닝\3-2_jester_recognation\jester_mediapipe_csv"

TARGET_LABELS = [
    "No gesture",
    "Doing other things",
    "Zooming In With Two Fingers",
    "Zooming Out With Two Fingers",
    "Zooming In With Full Hand",
    "Zooming Out With Full Hand",
    "Thumb Up",    
    "Thumb Down" 
]
def print_distribution(split_name):
    csv_path = os.path.join(DATA_ROOT, f"{split_name}_data.csv")
    
    if not os.path.exists(csv_path):
        print(f"{split_name} 파일을 찾을 수 없습니다: {csv_path}")
        return

    print(f"\n{split_name.upper()} 데이터 분포 확인 중...")
    
    try:
        df = pd.read_csv(csv_path, usecols=['video_id', 'label_name']) # 전체 데이터를 다 읽으면 느리므로, 필요한 컬럼만 로드
        
        counts = df.groupby('label_name')['video_id'].nunique() # 라벨별 고유한 video_id 개수 세기 (nunique)
        
        total_videos = 0
        print("-" * 50)
        print(f"{'Label Name':<30} | {'Video Count'}")
        print("-" * 50)
        
        for label in TARGET_LABELS:
            # 해당 라벨이 데이터에 없으면 0으로 처리
            count = counts.get(label, 0)
            print(f"{label:<30} | {count:5d}")
            total_videos += count
            
        print("-" * 50)
        print(f"{'Total':<30} | {total_videos:5d}")
        print("-" * 50)

    except Exception as e:
        print(f"파일 읽기 오류: {e}")

print_distribution('train')
print_distribution('val')


TRAIN 데이터 분포 확인 중...
--------------------------------------------------
Label Name                     | Video Count
--------------------------------------------------
No gesture                     |  1844
Doing other things             |  4374
Zooming In With Two Fingers    |  1801
Zooming Out With Two Fingers   |  1847
Zooming In With Full Hand      |  1799
Zooming Out With Full Hand     |  1832
Thumb Up                       |  1841
Thumb Down                     |  1810
--------------------------------------------------
Total                          | 17148
--------------------------------------------------

VAL 데이터 분포 확인 중...
--------------------------------------------------
Label Name                     | Video Count
--------------------------------------------------
No gesture                     |   256
Doing other things             |   713
Zooming In With Two Fingers    |   257
Zooming Out With Two Fingers   |   261
Zooming In With Full Hand      |   262
Zooming Out With

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from model import SE_TCN

DATA_ROOT = r"D:\학교\3-2\파이썬기반 딥러닝\3-2_jester_recognation\jester_mediapipe_csv"

TRAIN_CSV = os.path.join(DATA_ROOT, "train_data.csv")
SAVE_MODEL_PATH = "best_split_model.pth"
TARGET_LABELS = [
    "No gesture",
    "Doing other things",
    "Zooming In With Two Fingers",
    "Zooming Out With Two Fingers",
    "Zooming In With Full Hand",
    "Zooming Out With Full Hand",
    "Thumb Up",
    "Thumb Down"
]
LABEL_MAP = {label: i for i, label in enumerate(TARGET_LABELS)}
NUM_CLASSES = len(TARGET_LABELS)
VAL_SIZE = 0.2       
INPUT_CHANNELS = 126  # (21*3 + 21*3)
MAX_SEQ_LEN = 40
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 1e-3
PATIENCE = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class JesterRawDataset(Dataset): # 데이터셋 정의
    def __init__(self, csv_path):
        print(f"Loading {csv_path}...")
        self.df = pd.read_csv(csv_path)
        self.df = self.df[self.df['label_name'].isin(TARGET_LABELS)]
        self.grouped = self.df.groupby('video_id')
        self.video_ids = list(self.grouped.groups.keys())
        self.feat_cols = []
        for i in range(21):
            self.feat_cols.extend([f'joint_{i}_x', f'joint_{i}_y', f'joint_{i}_z'])
        self.labels = []
        for vid in self.video_ids:
            label_str = self.grouped.get_group(vid).iloc[0]['label_name']
            self.labels.append(LABEL_MAP[label_str])

    def __len__(self):
        return len(self.video_ids)

    def get_labels_by_indices(self, indices):
        return [self.labels[i] for i in indices]

    def __getitem__(self, idx):
        vid_id = self.video_ids[idx]
        group = self.grouped.get_group(vid_id)
        label_name = group.iloc[0]['label_name']
        label = LABEL_MAP[label_name]
        features = group[self.feat_cols].replace(0, np.nan).interpolate().fillna(0).values.astype(np.float32)
        return features, label


# Transforms(증강 기법 적용)
class RandomHorizontalFlip:
    def __init__(self, p=0.5):
        self.p = p

    def __call__(self, x):
        if random.random() < self.p:
            x = x.copy()
            x[:, 0::3] *= -1
        return x

class NormalizeSkeleton: # 정규화
    def __call__(self, x):
        T = x.shape[0]
        skeleton = x.reshape(T, 21, 3)
        wrist = skeleton[:, 0:1, :]
        skeleton = skeleton - wrist
        dist = np.linalg.norm(skeleton[:, 9, :] - skeleton[:, 0, :], axis=1, keepdims=True) + 1e-6
        skeleton = skeleton / dist[:, :, np.newaxis]
        return skeleton.reshape(T, -1)

class ComputeVelocity: # 속도 계산
    def __call__(self, x):
        velocity = np.zeros_like(x)
        velocity[1:] = x[1:] - x[:-1]
        combined = np.concatenate([x, velocity], axis=1)
        return combined

class ToTensorAndPad: # 제로 패딩
    def __init__(self, max_len=40):
        self.max_len = max_len

    def __call__(self, x):
        seq_len = x.shape[0]
        input_dim = x.shape[1]
        if seq_len < self.max_len:
            pad_len = self.max_len - seq_len
            padding = np.zeros((pad_len, input_dim), dtype=np.float32)
            x = np.vstack([x, padding])
        else:
            x = x[:self.max_len, :]
        x = x.transpose(1, 0)
        return torch.tensor(x, dtype=torch.float32)

class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, x):
        for t in self.transforms:
            x = t(x)
        return x

class HoloTouchCollate:
    def __init__(self, is_train=False):
        transforms_list = [NormalizeSkeleton(), ComputeVelocity()]
        if is_train:
            transforms_list.insert(0, RandomHorizontalFlip(p=0.5))
        transforms_list.append(ToTensorAndPad(MAX_SEQ_LEN))
        self.transform = Compose(transforms_list)

    def __call__(self, batch):
        inputs, targets = [], []
        for feature, label in batch:
            processed_feature = self.transform(feature)
            inputs.append(processed_feature)
            targets.append(label)
        return torch.stack(inputs), torch.tensor(targets, dtype=torch.long)

# 유틸리티 정의
class EarlyStopping:
    def __init__(self, patience=10, path=SAVE_MODEL_PATH):
        self.patience = patience
        self.path = path
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(model)

        elif val_loss > self.best_loss:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        torch.save(model.state_dict(), self.path)
        print(f"Best Model Saved! (Loss: {self.best_loss:.4f})")

def make_weighted_sampler_for_subset(dataset, indices):
    subset_labels = dataset.get_labels_by_indices(indices)
    class_counts = np.bincount(subset_labels, minlength=NUM_CLASSES)
    class_weights = 1. / (class_counts + 1e-6)
    sample_weights = [class_weights[l] for l in subset_labels]
    return WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

def main():
    print(f"Device: {DEVICE}")
    print("Starting Training with Train/Val Split...")

    full_dataset = JesterRawDataset(TRAIN_CSV)

    indices = np.arange(len(full_dataset))
    labels = full_dataset.labels
    train_idx, val_idx = train_test_split(
        indices,
        test_size=VAL_SIZE,
        stratify=labels,
        random_state=42
    )

    print(f"Data Split: Train {len(train_idx)} / Val {len(val_idx)}")

    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)

    train_sampler = make_weighted_sampler_for_subset(full_dataset, train_idx) # Train용 가중치 샘플러 (클래스 불균형 해소)

    # Train은 증강O, Val은 증강X
    train_collate = HoloTouchCollate(is_train=True)
    val_collate = HoloTouchCollate(is_train=False)

    train_loader = DataLoader(
        train_subset,
        batch_size=BATCH_SIZE,
        sampler=train_sampler,
        collate_fn=train_collate,
        num_workers=0
    )
    val_loader = DataLoader(
        val_subset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=val_collate,
        num_workers=0
    )

    model = SE_TCN(
        num_inputs=INPUT_CHANNELS,
        num_channels=[64, 64, 128, 128],
        num_classes=NUM_CLASSES
    ).to(DEVICE)

    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    early_stopping = EarlyStopping(patience=PATIENCE, path=SAVE_MODEL_PATH)

    best_acc = 0.0

    for epoch in range(EPOCHS):
        #  Train 
        model.train()
        train_loss = 0.0
        train_preds, train_targets_list = [], []

        loop = tqdm(train_loader, desc=f"Ep {epoch+1}/{EPOCHS}", leave=False)
        for inputs, targets in loop:

            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)     
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = nn.CrossEntropyLoss()(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_preds.extend(preds.cpu().numpy())
            train_targets_list.extend(targets.cpu().numpy())
            loop.set_postfix(loss=loss.item())

        train_loss /= len(train_loader)
        train_acc = accuracy_score(train_targets_list, train_preds)

        #  Val 
        model.eval()
        val_loss = 0.0
        val_preds, val_targets_list = [], []

        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                outputs = model(inputs)
                loss = nn.CrossEntropyLoss()(outputs, targets)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_targets_list.extend(targets.cpu().numpy())

        val_loss /= len(val_loader)
        val_acc = accuracy_score(val_targets_list, val_preds)

        # 결과 출력
        print(f"Ep {epoch+1} | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # 스케줄러 & Early Stopping
        scheduler.step(val_loss)
        early_stopping(val_loss, model)
 
        if val_acc > best_acc:
            best_acc = val_acc
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch+1}")
            break

    print("\n" + "="*40)
    print(f"Training Finished!")
    print(f"Best Val Accuracy: {best_acc:.4f}")
    print(f"Model saved to: {SAVE_MODEL_PATH}")
    print("="*40)

if __name__ == "__main__":
    main()

Device: cuda
Starting Training with Train/Val Split...
Loading D:\학교\3-2\파이썬기반 딥러닝\3-2_jester_recognation\jester_mediapipe_csv\train_data.csv...
Data Split: Train 13718 / Val 3430


d:\Program\anaconda3\envs\dl\lib\site-packages\torch\nn\utils\weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Ep 1 | Train Loss: 1.1957 Acc: 0.5571 | Val Loss: 1.0684 Acc: 0.6382
Best Model Saved! (Loss: 1.0684)


Ep 2 | Train Loss: 0.7542 Acc: 0.7717 | Val Loss: 0.7904 Acc: 0.7554
Best Model Saved! (Loss: 0.7904)


Ep 3 | Train Loss: 0.6308 Acc: 0.8169 | Val Loss: 0.6893 Acc: 0.7895
Best Model Saved! (Loss: 0.6893)


...

Ep 34 | Train Loss: 0.4089 Acc: 0.8736 | Val Loss: 0.5343 Acc: 0.8329
EarlyStopping counter: 6 out of 10


Ep 35 | Train Loss: 0.3942 Acc: 0.8799 | Val Loss: 0.5406 Acc: 0.8329
EarlyStopping counter: 7 out of 10


Ep 36 | Train Loss: 0.4081 Acc: 0.8726 | Val Loss: 0.5366 Acc: 0.8327
EarlyStopping counter: 8 out of 10


Ep 37 | Train Loss: 0.3817 Acc: 0.8822 | Val Loss: 0.5262 Acc: 0.8382
EarlyStopping counter: 9 out of 10


Ep 38 | Train Loss: 0.3750 Acc: 0.8851 | Val Loss: 0.5310 Acc: 0.8321
EarlyStopping counter: 10 out of 10
Early stopping at epoch 38

Training Finished!
Best Val Accuracy: 0.8397
Model saved to: best_split_model.pth


In [6]:
# convert_to_onnx
import torch.onnx
from model import SE_TCN
# ==========================================
# 1. 설정 (학습 때와 동일하게 맞춰야 함)
# ==========================================
MODEL_PATH = "best_split_model.pth"   # 변환할 PyTorch 모델 경로
ONNX_PATH = "HoloTouch_SE_TCN.onnx"   # 저장할 ONNX 파일 이름

# 모델 하이퍼파라미터
INPUT_CHANNELS = 126   # (21관절 * 3좌표) + (21관절 * 3속도)
NUM_CLASSES = 8        # 라벨 개수
MAX_SEQ_LEN = 40       # 시퀀스 길이
HIDDEN_CHANNELS = [64, 64, 128, 128] # 모델 구조

# ==========================================
# 2. 모델 로드 및 준비
# ==========================================
def convert():
    print(f"Loading model from {MODEL_PATH}...")
    
    # 모델 초기화 (CPU 모드로 로드 권장)
    device = torch.device('cpu')
    model = SE_TCN(
        num_inputs=INPUT_CHANNELS, 
        num_channels=HIDDEN_CHANNELS, 
        num_classes=NUM_CLASSES
    ).to(device)

    try:
        # 가중치 로드
        checkpoint = torch.load(MODEL_PATH, map_location=device)
        model.load_state_dict(checkpoint)
        print("PyTorch Model loaded successfully.")
    except Exception as e:
        print(f"Failed to load model: {e}")
        return

    # [중요] 평가 모드로 전환 (Dropout, BatchNorm 등의 동작 고정)
    model.eval()

    # ==========================================
    # 3. Dummy Input 생성 (모델 구조 파악용)
    # ==========================================
    # Shape: (Batch_Size, Channels, Seq_Len) -> (1, 126, 40)
    dummy_input = torch.randn(1, INPUT_CHANNELS, MAX_SEQ_LEN, requires_grad=True).to(device)

    # ==========================================
    # 4. ONNX Export 실행
    # ==========================================
    print(f"Exporting to {ONNX_PATH}...")
    
    torch.onnx.export(
        model,                      # 실행할 모델
        dummy_input,                # 모델 입력을 정의하는 더미 데이터
        ONNX_PATH,                  # 저장 경로
        export_params=True,         # 가중치 포함 여부
        opset_version=12,           # ONNX 버전 (11 or 12 권장)
        do_constant_folding=True,   # 상수 폴딩 최적화
        input_names=['input'],      # 입력 노드 이름 (나중에 추론할 때 씀)
        output_names=['output'],    # 출력 노드 이름
        dynamic_axes={              # 가변적인 차원 설정 (배치 사이즈 등)
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    )
    
    print(f"Conversion Complete! Saved as '{ONNX_PATH}'")
    
    # (선택) ONNX 파일 검증
    try:
        import onnx
        onnx_model = onnx.load(ONNX_PATH)
        onnx.checker.check_model(onnx_model)
        print("ONNX Model sanity check passed.")
    except ImportError:
        print("onnx' library not installed. Skipping validation.")
    except Exception as e:
        print(f"ONNX Model check failed: {e}")

if __name__ == "__main__":
    convert()

d:\Program\anaconda3\envs\dl\lib\site-packages\torch\nn\utils\weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_5864\3963470636.py:53: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1214 11:19:11.687000 5864 site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 12 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please c

Loading model from best_split_model.pth...
PyTorch Model loaded successfully.
Exporting to HoloTouch_SE_TCN.onnx...


W1214 11:19:12.190000 5864 site-packages\torch\onnx\_internal\exporter\_registration.py:107] torchvision is not installed. Skipping torchvision::nms


[torch.onnx] Obtain model graph for `SE_TCN([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `SE_TCN([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 12).
Failed to convert the model to the target version 12 using the ONNX C API. The model was not modified
Traceback (most recent call last):
  File "d:\Program\anaconda3\envs\dl\lib\site-packages\onnxscript\version_converter\__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
  File "d:\Program\anaconda3\envs\dl\lib\site-packages\onnxscript\version_converter\_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
  File "d:\Program\anaconda3\envs\dl\lib\site-packages\onnxscript\version_converter\__init__.py", line 122, in _partial_convert_version
    return onnx.version_converter.convert_version(
  File "d:\Program\anaconda3\envs\dl\lib\site-packages\onnx\version_converter.py", line 39, in convert_version
    converted_model_str = C.convert_version(model_str, target_versio

[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 20 of general pattern rewrite rules.
Conversion Complete! Saved as 'HoloTouch_SE_TCN.onnx'
ONNX Model sanity check passed.


In [1]:
# ==========================================
# [수정됨] ONNX 변환 코드 (weight_norm 제거 포함)
# ==========================================
import torch
import torch.nn as nn
from torch.nn.utils import remove_weight_norm
from model import SE_TCN

# 1. 설정
MODEL_PATH = "best_split_model.pth"   
ONNX_PATH = "HoloTouch_SE_TCN_Fixed.onnx" # 이름 변경 권장

# 모델 하이퍼파라미터 (학습 때와 동일하게)
INPUT_CHANNELS = 126
NUM_CLASSES = 8
HIDDEN_CHANNELS = [64, 64, 128, 128]

# 2. weight_norm 제거 함수 (★ 핵심 ★)
# TCN 모델은 weight_norm을 쓰는데, 이게 ONNX/Sentis에서 버그를 자주 일으킵니다.
# 변환 전에 이를 제거하여 순수 Convolution 가중치로 고정해야 합니다.
def remove_norm(module):
    for name, child in module.named_children():
        if isinstance(child, (nn.Conv1d, nn.Linear)):
            # weight_norm이 적용된 레이어인지 확인
            if hasattr(child, 'weight_g'): 
                remove_weight_norm(child)
                print(f"Removed weight_norm from {name}")
        else:
            remove_norm(child)

def convert():
    print(f"Loading model from {MODEL_PATH}...")
    
    device = torch.device('cpu') # 변환은 CPU에서 하는 것이 안전함
    model = SE_TCN(
        num_inputs=INPUT_CHANNELS, 
        num_channels=HIDDEN_CHANNELS, 
        num_classes=NUM_CLASSES
    ).to(device)

    try:
        checkpoint = torch.load(MODEL_PATH, map_location=device)
        model.load_state_dict(checkpoint)
        print("PyTorch Model loaded successfully.")
    except Exception as e:
        print(f"Failed to load model: {e}")
        return

    # [중요 1] 평가 모드
    model.eval()

    # [중요 2] weight_norm 제거 (Sentis 호환성 확보)
    print("Removing weight_norm for Sentis compatibility...")
    remove_norm(model)

    # 3. Dummy Input (배치 1, 채널 126, 시퀀스 40)
    dummy_input = torch.randn(1, INPUT_CHANNELS, 40, requires_grad=False).to(device)

    # 4. ONNX Export
    print(f"Exporting to {ONNX_PATH}...")
    torch.onnx.export(
        model,
        dummy_input,
        ONNX_PATH,
        export_params=True,
        opset_version=12,           # Sentis 권장 버전
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    )
    print(f"Conversion Complete! Saved as '{ONNX_PATH}'")

if __name__ == "__main__":
    convert()

Loading model from best_split_model.pth...
PyTorch Model loaded successfully.
Removing weight_norm for Sentis compatibility...
Removed weight_norm from conv1
Removed weight_norm from conv2
Removed weight_norm from conv1
Removed weight_norm from conv2
Removed weight_norm from conv1
Removed weight_norm from conv2
Removed weight_norm from conv1
Removed weight_norm from conv2
Exporting to HoloTouch_SE_TCN_Fixed.onnx...


d:\Program\anaconda3\envs\dl\lib\site-packages\torch\nn\utils\weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39904\1301154849.py:61: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1214 16:44:51.693000 39904 site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 12 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please

[torch.onnx] Obtain model graph for `SE_TCN([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `SE_TCN([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 12).
Failed to convert the model to the target version 12 using the ONNX C API. The model was not modified
Traceback (most recent call last):
  File "d:\Program\anaconda3\envs\dl\lib\site-packages\onnxscript\version_converter\__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
  File "d:\Program\anaconda3\envs\dl\lib\site-packages\onnxscript\version_converter\_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
  File "d:\Program\anaconda3\envs\dl\lib\site-packages\onnxscript\version_converter\__init__.py", line 122, in _partial_convert_version
    return onnx.version_converter.convert_version(
  File "d:\Program\anaconda3\envs\dl\lib\site-packages\onnx\version_converter.py", line 39, in convert_version
    converted_model_str = C.convert_version(model_str, target_versio

[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 20 of general pattern rewrite rules.
Conversion Complete! Saved as 'HoloTouch_SE_TCN_Fixed.onnx'
